In [1]:
import numpy as np

In [22]:
# nameValue：节点的名字
# numOccur：该节点的计数值
# parentNode：父节点的名字
class treeNode:
    # name：节点名字
    # count：计数值
    # nodeLink: 虚线，连接相同元素项
    # parent：当前节点的父节点，方便上溯
    # children：当前节点的子节点，也是treeNode类型
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None
        self.parent = parentNode
        self.children = {}
    # 对count变量进行更新
    def inc(self, numOccur):
        self.count += numOccur
    # display函数，将树以文本方式展示，调试用，对于构建树不是必要的
    def disp(self, ind=1):
        # 通过‘-’数量缩进来控制层级
        print('-'*ind, self.name, ' ', self.count)
        # child：class treeNode
        for child in self.children.values():
            child.disp(ind + 1)

In [16]:
# 原始数据生成函数
def loadSimpDat():
    simpDat = [['r','z','h','j','p'],
               ['z','y','x','w','v','u','t','s'],
               ['z'],
               ['r','x','n','o','s'],
               ['y','r','x','z','q','t','p'],
               ['y','z','x','e','q','s','t','m']]
    return simpDat  

# 遍历 rawData，生成便于生成 FP树的输入数据（字典格式）
# return {frozenset({trans}):count}
def createInitSet(simpDat):
    retDict = {}
    for trans in simpDat:
        try:
            retDict[frozenset(trans)] += 1
        except KeyError:
            retDict[frozenset(trans)] = 1
    return retDict

In [18]:
simpDat = loadSimpDat()
initSet = createInitSet(simpDat)
initSet

{frozenset({'z'}): 1,
 frozenset({'h', 'j', 'p', 'r', 'z'}): 1,
 frozenset({'s', 't', 'u', 'v', 'w', 'x', 'y', 'z'}): 1,
 frozenset({'n', 'o', 'r', 's', 'x'}): 1,
 frozenset({'p', 'q', 'r', 't', 'x', 'y', 'z'}): 1,
 frozenset({'e', 'm', 'q', 's', 't', 'x', 'y', 'z'}): 1}

In [79]:
# 增长相同元素指针链，延长 nodelink 指针
# nodeToTest：指针测试的起始点
# targetNode：指针的终点
# return NULL 过程函数，更新头指针表
def updateHeader(nodeToTest, targetNode):
    while nodeToTest.nodeLink != None:
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

In [86]:
# items：某事务中的元素（一阶频繁项）列表，按照其在头指针表的频次统计降序排列
# inTree：FP树，treeNode类
# headerTable：头指针表
# count：该元素所在事务发生的次数
# return NULL 过程函数，对树和头指针表进行更新操作，不返回值
def updateTree(items, inTree, headerTable, count):
    # 如果打头（频次最大）的那个元素存在树的子节点中，则把该事务的出现频次全都累加到该子节点上
    if(items[0] in inTree.children):
        inTree.children[items[0]].inc(count)
    # 如果在子节点中还没有这个元素，那么在树上创建这个子节点，同时指定父节点为当前的 inTree
    else:
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        # 刚生成的新节点需要与之前已经在树里的其他分支中的相同元素进行连接
        # 这是一条串链，只会不断变长
        # 如果最大元素的相同元素连接信息（待添加的nodelink）为空，则更新为当前节点
        if headerTable[items[0]][1] == None:
            headerTable[items[0]][1] = inTree.children[items[0]]
        # 如果连接信息不为空，则不更改头指针表
        # 沿着头指针表给出的连接节点，根据其nodelink一直追溯到串链的尾部。
        # 则在串链的终点定位为当前（把当前节点纳入到串中）
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
    # 对剩下的元素，递归地生成树，其中指定树是以上一次迭代的 items[0]为根节点的。
    if len(items)>1:
        updateTree(items[1::], inTree.children[items[0]], headerTable, count)

In [83]:
# dataSet：Dict类型 {frozenset({trans}):count}
# 根据各元素的统计数量决定了谁趋向于在顶部，谁趋向于在底部
def createTree(dataSet, minSup=1):
    # 一、统计筛选部分
    # 第一次遍历数据集，统计各个元素的发生频次
    headerTable = {}
    # 默认 trans是key
    for trans in dataSet.keys():
        for item in trans:
            # dataSet[trans]是对该事务发生频次的统计 count
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]
    # 对收集到的所有键进行遍历，做两件事
    # 1、如果某个元素的频次小于最小频次要求，则在头指针表中删除此键
    # 2、给键值增加元素，相同元素的起始指针
    # headerTable： {"item“：[count,None]}
    keys = list(headerTable.keys()) # 必须提前以列表形式缓存 headerTable的 keys
    for key in keys:
        if headerTable[key] < minSup:
            del headerTable[key]
        # 初始化头指针表中相同元素的起始指针，如果不是None的话，就treeNode类型
        else:
            headerTable[key] = [headerTable[key], None]
    # 汇总满足要求的所有元素（一阶频繁项）
    freqItemSet = set(headerTable.keys())
    # 如果没有满足条件的元素，则退出
    if len(freqItemSet)==0: return None, None    
    
    # 二、建立 FP 树部分
    # 初始化 FP树，建立根节点
    retTree = treeNode('Null Set', 1, None)
    # 第二次遍历数据集
    # 将每个事务的元素都纳入到树中，保存其相互关联的信息
    for trans, count in dataSet.items():
        localD = {}
        for item in trans:
            if item in freqItemSet:
                localD[item] = headerTable[item][0]
        if len(localD) > 0:
            # 对收集的元素，按照统计频次降序排列
            sortedItems = sorted(localD.items(), key=lambda x: x[1], reverse=True)
            # 按照已经降序排列的统计元素，只拿其键值（形成一个单独事务的元素列表）
            sortedKeys = [v[0] for v in sortedItems]
            # 将该事务内的元素放入树中
            updateTree(sortedKeys, retTree, headerTable, count)
    return retTree, headerTable            

In [84]:
simpData = loadSimpDat()
initSet = createInitSet(simpData)

In [87]:
fpTree, headerTable = createTree(initSet, minSup=3)

In [111]:
# 上溯整棵树至根部
# 在此过程中不断累加前缀（倒序的，从底部到顶部）
def ascendTree(leafNode, prefixPath):
    if leafNode.parent != None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent, prefixPath)
# 根据 basePat，获取所有前缀及事物数量
# basePat： 模式基，即不同数量元素组成的模式
# headerTable：已经伴随 FP树生成的头指针表
def findPrefixPath(basePat, headerTable):
    condPats = {}
    # FP 树中该 basePat的串链头
    treeNode = headerTable[basePat][1]
    # 从串链头开始循环向下寻找前缀和数量
    while treeNode != None:
        prefixPath = []
        ascendTree(treeNode, prefixPath)
        if len(prefixPath) > 1:
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        # 寻找在其他分支内相同的元素
        treeNode = treeNode.nodeLink
    return condPats

In [112]:
findPrefixPath("r", headerTable)

{frozenset({'z'}): 1, frozenset({'x'}): 1, frozenset({'t', 'x', 'z'}): 1}

In [160]:
freqItemList=[]

In [204]:
def mineTree(inTree, headerTable, minSup, preFix=set([]), freqItemList=[],i=0):
    # 从底层元素开始依次往上
    sortedItem = sorted(headerTable.items(), key=lambda x: x[1][0])
    bigL = [v[0] for v in sortedItem]
    for basePat in bigL:
        newFreqSet = preFix.copy()
        # 随着递归的进行，这个集合不断地累加，意味着 basePat变得复杂
        newFreqSet.add(basePat)
        print("newFreqSet:", newFreqSet)
        # 记录多次生成的频繁集, 一个容器
        freqItemList.append(newFreqSet)
        # 根据当前条件基，找到所有前缀及事务频次，即生成条件事务集
        condPattBases = findPrefixPath(basePat, headerTable)
        print(len(condPattBases))
        # 利用条件事务集可以生成新的 FP树（条件 FP树）
        myCondTree, myHead = createTree(condPattBases, minSup)
        # 如果仍然可以有树生成，则递归地
        if myHead != None:
            print('conditional tree for: ', newFreqSet, "layer:",i)
            myCondTree.disp()
            mineTree(myCondTree, myHead, minSup, newFreqSet, freqItemList,i=i+1)
        else:
            print("myHead is none")

In [205]:
mineTree(fpTree, headerTable, minSup=3)

newFreqSet: {'r'}
3
myHead is none
newFreqSet: {'t'}
1
conditional tree for:  {'t'} layer: 0
- Null Set   1
-- x   3
--- z   3
newFreqSet: {'t', 'x'}
0
myHead is none
newFreqSet: {'t', 'z'}
1
conditional tree for:  {'t', 'z'} layer: 1
- Null Set   1
-- x   3
newFreqSet: {'t', 'x', 'z'}
0
myHead is none
newFreqSet: {'s'}
2
conditional tree for:  {'s'} layer: 0
- Null Set   1
-- x   3
newFreqSet: {'s', 'x'}
0
myHead is none
newFreqSet: {'y'}
2
conditional tree for:  {'y'} layer: 0
- Null Set   1
-- t   3
--- x   3
---- z   3
newFreqSet: {'t', 'y'}
0
myHead is none
newFreqSet: {'y', 'x'}
1
conditional tree for:  {'y', 'x'} layer: 1
- Null Set   1
-- t   3
newFreqSet: {'t', 'y', 'x'}
0
myHead is none
newFreqSet: {'y', 'z'}
1
conditional tree for:  {'y', 'z'} layer: 1
- Null Set   1
-- t   3
--- x   3
newFreqSet: {'t', 'y', 'z'}
0
myHead is none
newFreqSet: {'y', 'x', 'z'}
1
conditional tree for:  {'y', 'x', 'z'} layer: 2
- Null Set   1
-- t   3
newFreqSet: {'t', 'y', 'x', 'z'}
0
myHead is 

In [181]:
basePat = 'z'
minSup = 3
preFix= newFreqSet
newFreqSet = preFix.copy()
newFreqSet.add(basePat)
freqItemList.append(newFreqSet)

In [182]:
newFreqSet

{'t', 'x', 'z'}

In [183]:
freqItemList

[{'t'}, {'t', 'x'}, {'t', 'x', 'z'}]

In [184]:
condPattBases = findPrefixPath(basePat, headerTable)
condPattBases

{}

In [185]:
myCondTree, myHead = createTree(condPattBases, minSup)

In [186]:
myCondTree.disp()

AttributeError: 'NoneType' object has no attribute 'disp'

In [188]:
myHead == None

True

In [180]:
sortedItem = sorted(myHead.items(), key=lambda x: x[1][0])
bigL = [v[0] for v in sortedItem]
bigL

['z']